<a href="https://colab.research.google.com/github/chitransh0052/Gen-AI-Workshop/blob/master/Gen_AI_Day2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**TRANSFORMER**

In [ ]:
import torch
import torch.nn as nn
import math

In [ ]:
#positional encoding

class posittionEncoding(nn.Module):

  def _init_(self,d_model,max_seq_len=8):

    super()._init_()

    # create an empty tensir to store positional encoding

    pe=torch.zeros(max_seq_len,d_model)

    #genrating position indicies(0,1,2,3,..)

    pos=torch.arange(0,max_seq_len,dtype=torch.float).unsqueeze(1)

    #genratee division term (1000^(2i/d_model))

    div_term=torch.exp(torch.arange(0,d_model,2).float()*(-math.log(10000.0)/d_model))

    #apply sine top even indices of embeding dimensions

    pe[:,0::2]=torch .sin(pos*div_term)

    pe[:,1::2]=torch .cos(pos*div_term)

    #add bash dimesions

    self.pe=pe.unsqueeze(0)

  def forward(self,x):

    #add x shape:batch size ,seq_len,d_model

    #add postitional encoding to embeding

    return x+self.pe[:,:x.size(1)].to(x.device)

#transformer enncoding block

class TransformerEncoderBlock(nn.Module):

  def _init_(self,d_model,heads,ff_dim,dropout=0.1):

    super()._init_()

    #multiheaded attention layer

    self.attn=nn.MultiheadAttention(d_model,heads,dropout=dropout,batch_first=True)

    #feedforward network to liner layer + relu

    self.ff=nn.Sequential(

        nn.Linear(d_model,ff_dim),

        nn.ReLU(),

        nn.Linear(ff_dim,d_model)

    )

    #layer normal;isation

    self.norm1=nn.LayerNorm(d_model)

    self.norm2=nn.LayerNorm(d_model)

    #dropout

    self.dropout=nn.Dropout(dropout)

  def forward(self,x):

    #multiheaded attention

    attn_out,_=self.attn(x,x,x)#qurery, key,value are the self attention parameters.

    #add an norm (residual connection)

    x=self.norm1(x+self.dropout(attn_out))

    #feedforward network layer

    ff_out=self.ff(x)

    #add an norm (residual connection)

    x=self.norm2(x+self.dropout(ff_out))



    return x

In [ ]:
# Transformer Architecture (Encoder Only)
import torch
import torch.nn as nn
import math

# Positional Encoding to inject position information into token embeddings
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=5000):
        super().__init__()

        # Create a matrix of shape (max_len, d_model) to hold position encodings
        pe = torch.zeros(max_len, d_model)

        # Position indices (0 to max_len-1)
        pos = torch.arange(0, max_len).unsqueeze(1).float()

        # Compute the scaling factors for sine and cosine (based on d_model)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))

        # Apply sine to even indices in the embedding dimension
        pe[:, 0::2] = torch.sin(pos * div_term)

        # Apply cosine to odd indices in the embedding dimension
        pe[:, 1::2] = torch.cos(pos * div_term)

        # Add a batch dimension (1, max_len, d_model)
        self.pe = pe.unsqueeze(0)

    def forward(self, x):
        # Add position encoding to input embeddings
        # x: (batch_size, seq_len, d_model)
        return x + self.pe[:, :x.size(1)].to(x.device)

# A single Transformer Encoder Block
class TransformerEncoderBlock(nn.Module):
    def __init__(self, d_model, heads, ff_dim, dropout=0.1):
        super().__init__()

        # Multi-head self-attention (batch_first=True => input shape is (batch, seq, features))
        self.attn = nn.MultiheadAttention(d_model, heads, dropout=dropout, batch_first=True)

        # Feed-forward network: Linear -> ReLU -> Linear
        self.ff = nn.Sequential(
            nn.Linear(d_model, ff_dim),
            nn.ReLU(),
            nn.Linear(ff_dim, d_model),
        )

        # Layer normalization (prevents vanishing/exploding gradients)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)

        # Dropout for regularization
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        # Self-attention layer
        attn_out, _ = self.attn(x, x, x)

        # Add & Norm (residual connection + layer norm)
        x = self.norm1(x + self.dropout(attn_out))

        # Feed-forward layer
        ff_out = self.ff(x)

        # Add & Norm again
        x = self.norm2(x + self.dropout(ff_out))
        return x

# Full Transformer Encoder model
class SimpleTransformerEncoder(nn.Module):
    def __init__(self, vocab_size, d_model=128, nhead=8, ff_dim=512, num_layers=2, max_len=100):
        super().__init__()

        # Token embedding layer
        self.embed = nn.Embedding(vocab_size, d_model)

        # Positional encoding
        self.pos_enc = PositionalEncoding(d_model, max_len)

        # Stack multiple Transformer encoder blocks
        self.layers = nn.ModuleList([
            TransformerEncoderBlock(d_model, nhead, ff_dim) for _ in range(num_layers)
        ])

        # Final output layer to map to vocab size (e.g., for language modeling)
        self.fc_out = nn.Linear(d_model, vocab_size)

    def forward(self, x):
        # Input shape: (batch_size, sequence_length)

        # Convert token IDs to embeddings: (batch_size, sequence_length, d_model)
        x = self.embed(x)

        # Add positional encoding
        x = self.pos_enc(x)

        # Pass through each encoder block
        for layer in self.layers:
            x = layer(x)

        # Output logits for each position: (batch_size, sequence_length, vocab_size)
        return self.fc_out(x)


model = SimpleTransformerEncoder(vocab_size=10000)
input_tensor = torch.randint(0, 10000, (2, 20))  # (batch_size=2, sequence_length=20)
output = model(input_tensor)
print(output.shape)

torch.Size([2, 20, 10000])
